In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lr.text_processing.util import pre_process_nli_df
from lr.training.util import get_ternary_label, filter_df_by_label
from lr.text_processing.transformations.wordnet import path_base_transformation

from sklearn.exceptions import ConvergenceWarning
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)

### Loading data

In [2]:
train_path = "data/snli/train.csv"
dev_path = "data/snli/dev.csv"

train_path_mod = "data/snli/train_p_h_syn_noun.csv"
dev_path_mod = "data/snli/dev_p_h_syn_noun.csv"

train_trans = lambda df: path_base_transformation(df, train_path_mod)
dev_trans = lambda df: path_base_transformation(df, dev_path_mod)


train = pd.read_csv(train_path)
dev = pd.read_csv(dev_path)
train = filter_df_by_label(train.dropna()).reset_index(drop=True)
dev = filter_df_by_label(dev.dropna()).reset_index(drop=True)

train = train.head(10000)
dev = dev.head(1000)

dev_t = dev_trans(dev)

pre_process_nli_df(train)
pre_process_nli_df(dev)

print(train.shape)
print(dev.shape)

(10000, 3)
(1000, 3)


### Params

In [3]:
from lr.training.language_representation import Tfidf
from lr.training.util import get_ternary_label

max_features = None

param_grid = {"C": np.linspace(0, 3, 50),
              "penalty":["l2"]}


hyperparams = {"RepresentationFunction": Tfidf,
               "cv":5,
               "solver": 'lbfgs',
               "verbose":True,
               "n_jobs":1,
               "n_iter":2,
               "max_features": max_features,
               "label_translation": get_ternary_label,
               "param_grid": param_grid}


## DGP

In [4]:
from lr.stats.h_testing import DGP

dgp_rs = [24]

dgp = DGP(train, train_trans, rho=0.3)
train_ = dgp.sample(random_state=dgp_rs[0])

## Train different models on the same sample

In [5]:
from lr.models.logistic_regression import LRWrapper


training_rs = [34, 45]
models = []

for rs in training_rs:
    hyperparams["random_state"] = rs
    lr = LRWrapper(hyperparams)
    lr.fit(train_)
    models.append(lr)


Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   20.0s finished


Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   12.7s finished


### Get majority vote

In [6]:
from lr.stats.h_testing import  Ensemble

accs = [lr.get_acc(train_) for lr in models]
lr = Ensemble(models)

print("mean acc = {:.3f} | std = {:.3f}".format(np.mean(accs), np.std(accs)))
print("majority vote acc = {:.5f}".format(lr.get_acc(train_)))


mean acc = 0.634 | std = 0.022
majority vote acc = 0.63800


### Get t statistic using majority model and dev data

In [7]:
from lr.stats.h_testing import  get_matched_results, get_paired_t_statistic

results = get_matched_results(dev, dev_t, lr, lr.label_translation)
t_obs = get_paired_t_statistic(results)

print(t_obs)

-1.3847661716071549


In [8]:
## -1.3847661716071549